In [1]:
import tensorflow as tf
import math
from IPython import display
import numpy as np
import pandas as pd
from sklearn import metrics
from matplotlib import pyplot as plt
from matplotlib import gridspec
from matplotlib import cm
from tensorflow.python.data import Dataset
tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format
print("Completed")

Completed


In [2]:
california_housing_dataframe = pd.read_csv('/Users/biplabkumardas/google-tensorflow/california_housing_train.csv', sep=",")
print("Completed")

Completed


In [3]:
california_housing_dataframe = california_housing_dataframe.reindex(
    np.random.permutation(california_housing_dataframe.index))
california_housing_dataframe["median_house_value"] /= 1000.0
california_housing_dataframe

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
10667,-120.6,35.1,15.0,5818.0,924.0,2324.0,845.0,4.4,267.6
16609,-122.7,38.4,38.0,1689.0,526.0,1071.0,529.0,1.5,124.0
16114,-122.5,37.8,52.0,1817.0,449.0,948.0,380.0,3.9,390.0
2729,-117.7,33.6,17.0,2150.0,361.0,1194.0,335.0,5.5,227.0
16137,-122.5,37.7,39.0,2076.0,482.0,1738.0,445.0,3.2,232.1
...,...,...,...,...,...,...,...,...,...
13851,-122.0,37.4,43.0,1261.0,317.0,836.0,333.0,4.1,224.6
1172,-117.1,33.1,17.0,3164.0,652.0,1123.0,699.0,2.1,80.0
8780,-118.6,34.3,15.0,10860.0,1653.0,4178.0,1581.0,6.3,262.1
11614,-121.3,37.8,32.0,696.0,151.0,443.0,144.0,2.5,86.3


In [4]:
california_housing_dataframe.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0
mean,-119.6,35.6,28.6,2643.7,539.4,1429.6,501.2,3.9,207.3
std,2.0,2.1,12.6,2179.9,421.5,1147.9,384.5,1.9,116.0
min,-124.3,32.5,1.0,2.0,1.0,3.0,1.0,0.5,15.0
25%,-121.8,33.9,18.0,1462.0,297.0,790.0,282.0,2.6,119.4
50%,-118.5,34.2,29.0,2127.0,434.0,1167.0,409.0,3.5,180.4
75%,-118.0,37.7,37.0,3151.2,648.2,1721.0,605.2,4.8,265.0
max,-114.3,42.0,52.0,37937.0,6445.0,35682.0,6082.0,15.0,500.0


In [12]:
## getting the feature and ground truth or the labels associated with 
my_feature = california_housing_dataframe[['total_rooms']]
feature_columns = [tf.feature_column.numeric_column("total_rooms")]
print(my_feature)
print(feature_columns)

       total_rooms
10667       5818.0
16609       1689.0
16114       1817.0
2729        2150.0
16137       2076.0
...            ...
13851       1261.0
1172        3164.0
8780       10860.0
11614        696.0
14902       1303.0

[17000 rows x 1 columns]
[_NumericColumn(key='total_rooms', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [13]:
## define the labels
targets = california_housing_dataframe["median_house_value"]
print(targets)

10667   267.6
16609   124.0
16114   390.0
2729    227.0
16137   232.1
         ... 
13851   224.6
1172     80.0
8780    262.1
11614    86.3
14902    92.8
Name: median_house_value, Length: 17000, dtype: float64


In [16]:
## define the optimizer
my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0000001)
my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)

linear_regressor = tf.estimator.LinearRegressor(
    feature_columns = feature_columns,
    optimizer = my_optimizer
)

print("Completed")

Completed


In [18]:
# preprocessing of the data
def my_input_fn(features, batch_size = 1, shuffle = True, num_epochs=None):
    features = {key:np.array(value) for key, value in dict(features).items()}
    ds = Dataset.from_tensor_slices((features, targets))
    ds = ds.batch(batch_size).repeat(num_epochs)
    if shuffle:
        ds = ds.shuffle(buffer_size=10000)
        features, labels = ds.make_one_shot_iterator().get_next()
        return features, labels

print("Completed")

Completed
